# KNN

## Variantes


- RadiusNeighborsClassifier: In cases where the data is not uniformly sampled. The user specifies a fixed radius r, such that points in sparser neighborhoods use fewer nearest neighbors for the classification. For high-dimensional parameter spaces, this method becomes less effective due to the so-called “curse of dimensionality”.
- KNeighborsClassifier: implements learning based on the k nearest neighbors of each query point, where k is an integer value specified by the user. In general, a larger k suppresses the effects of noise, but makes the classification boundaries less distinct.

# sklearn.neighbors.DistanceMetric


Metrics intended for integer-valued vector spaces: Though intended for integer-valued vectors, these are also valid metrics in the case of real-valued vectors.
identifier	class name	distance function
“hamming”	HammingDistance	N_unequal(x, y) / N_tot
No tan beneficioso: que si tenemos un dato donde aparece 10 veces la palabra 'viagra' y otro donde aparece 11 veces, no lo considera como cercano. (Por ahi tengamos que revisar los datos y separar en categorias: 'no aparece', 'aparece hasta 3 veces' y 'aparece mas de 3')
“canberra”	CanberraDistance	sum(|x - y| / (|x| + |y|))
“braycurtis”	BrayCurtisDistance	sum(|x - y|) / (sum(|x|) + sum(|y|))


En el caso que lleguemos a cambiar a los vectores por vectores booleanos: (no las mire, solo copie porque no creo que hagamos esto.)

Metrics intended for boolean-valued vector spaces: Any nonzero entry is evaluated to “True”. In the listings below, the following abbreviations are used:
N : number of dimensions
NTT : number of dims in which both values are True
NTF : number of dims in which the first value is True, second is False
NFT : number of dims in which the first value is False, second is True
NFF : number of dims in which both values are False
NNEQ : number of non-equal dimensions, NNEQ = NTF + NFT
NNZ : number of nonzero dimensions, NNZ = NTF + NFT + NTT
identifier	class name	distance function
“jaccard”	JaccardDistance	NNEQ / NNZ
“maching”	MatchingDistance	NNEQ / N
“dice”	DiceDistance	NNEQ / (NTT + NNZ)
“kulsinski”	KulsinskiDistance	(NNEQ + N - NTT) / (NNEQ + N)
“rogerstanimoto”	RogersTanimotoDistance	2 * NNEQ / (N + NNEQ)
“russellrao”	RussellRaoDistance	NNZ / N
“sokalmichener”	SokalMichenerDistance	2 * NNEQ / (N + NNEQ)
“sokalsneath”	SokalSneathDistance	NNEQ / (NNEQ + 0.5 * NTT)




Algo que se me ocurrio es que podemos hacer es mezclar un poco, separar el data frame entre los atributos booleanos y los que no lo son. Hacer KNN con ambos datos por separado y despues unir los resultados con algun peso para cada dato, pero no se. Lo vemos.

In [1]:
# Esto agrega al python path el directorio ..
import sys
sys.path.append("..")

import numpy as np
import config


from dataframe_builder import DataFrameBuilder
# Ignorar Warning de sklearn (no es importante)
import warnings
warnings.filterwarnings('ignore',
                        message='Changing the shape of non-C contiguous array')


print "Creando dataframe..."


builder = DataFrameBuilder()
df = builder.build()

print "Listo"

Creando dataframe...
Buscando dataframe en C:\Users\okus\Documents\GitHub\tp-aa\cache\dataframe.pickle
Encontrado. Dimensiones: (90000, 56)
Listo


# Optimización de hiperparámetros

Busquemos los mejores (posibles) hiperparámetros

Para eso, primero veamos qué hiperparámetros nos provee la implementación de SKLearn.

In [4]:
from sklearn.neighbors import KNeighborsClassifier

help(KNeighborsClassifier)

Help on class KNeighborsClassifier in module sklearn.neighbors.classification:

class KNeighborsClassifier(sklearn.neighbors.base.NeighborsBase, sklearn.neighbors.base.KNeighborsMixin, sklearn.neighbors.base.SupervisedIntegerMixin, sklearn.base.ClassifierMixin)
 |  Classifier implementing the k-nearest neighbors vote.
 |  
 |  Read more in the :ref:`User Guide <classification>`.
 |  
 |  Parameters
 |  ----------
 |  n_neighbors : int, optional (default = 5)
 |      Number of neighbors to use by default for :meth:`k_neighbors` queries.
 |  
 |  weights : str or callable
 |      weight function used in prediction.  Possible values:
 |  
 |      - 'uniform' : uniform weights.  All points in each neighborhood
 |        are weighted equally.
 |      - 'distance' : weight points by the inverse of their distance.
 |        in this case, closer neighbors of a query point will have a
 |        greater influence than neighbors which are further away.
 |      - [callable] : a user-defined functi

# En clase Agustin dijo :

- Técnica simple que a veces permite aproximar conceptos muy complejos.
- El entrenamiento es muy rápido. (¿Entrenamiento?)
- La consulta es muy lenta. (AyED eficientes.)
- El modelo (¿modelo?) ocupa mucho espacio en disco.
- Para pensar: La distancia se calcula con todos los atributos. ¿Qué pasa si algunos son irrelevantes?

Lo unico que se me ocurrio para responder es que se podra ahorrar bastante tiempo si no consideramos los atributos que estan al pepe. Para el informe estaria bueno volver a tirar knn despues de haber hecho pca o ica sobre los atributos y poner un timer a ver que onda.
Uds piensan que sera eso?

Opciones:

- n_neighbors: Cantidad de vecinos (?)
- weights: str
- splitter: No estoy muy seguro
- max_depth: máxima profundidad del árbol
- max_features: qué porcentaje de variables tomo a la hora de partir un nodo.
  Estas variables se eligen aleatoriamente
- min_samples_split: cuántos elementos tengo que tener en un nodo para decidir
  partirlo

In [1]:
# Esto agrega al python path el directorio ..
import sys
sys.path.append("..")

import numpy as np
import config
from dataframe_builder import DataFrameBuilder

builder = DataFrameBuilder()
df = builder.build()

Buscando dataframe en C:\Users\okus\Documents\GitHub\tp-aa\cache\dataframe.pickle
Encontrado. Dimensiones: (80995, 72)
trate de borrar pero no pude
Dataframe construído
Dataframe guardado en C:\Users\okus\Documents\GitHub\tp-aa\cache\dataframe.pickle
Dimensiones: (80995, 72)


In [5]:
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from sklearn.cross_validation import cross_val_score

#warnings.filterwarnings('ignore', category=DeprecationWarning)

options = {
    'n_neighbors': range(1000, 1002),
    'weights': ['distance'],
    'metric' : ['canberra', 'braycurtis']
    
}

# Preparo data para clasificar
X = df.design_matrix
y = df.outcomes

scoring_methods = ['precision', 'accuracy', 'f1', 'recall', 'roc_auc']

for scoring in scoring_methods:
    clf = KNeighborsClassifier()

    print("=" * 80 + "\n")
    print("Scoring {}".format(scoring))

    #search = GridSearchCV(clf, scoring=scoring, param_grid=options, n_jobs=4)
    search = RandomizedSearchCV(clf, scoring=scoring, param_distributions=options, n_jobs=4, n_iter=40)
    
    search.fit(X, y)
    
    print "Mejor combinación: {}".format(search.best_params_)
    print "Mejor valor: {}\n\n".format(search.best_score_)

        
    

    


Scoring precision


ValueError: The total space of parameters 4 is smaller than n_iter=40. For exhaustive searches, use GridSearchCV.

In [5]:
from sklearn.neighbors import KNeighborsClassifier, DistanceMetric
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from sklearn.cross_validation import cross_val_score




# Preparo data para clasificar
X = df.design_matrix
y = df.outcomes



    #search = GridSearchCV(clf, scoring=scoring, param_grid=options, n_jobs=4)
    #search = RandomizedSearchCV(clf, scoring=scoring, param_distributions=options, n_jobs=4, n_iter=40)
search= KNeighborsClassifier(n_neighbors=10, algorithm='ball_tree',weights= 'distance', metric='braycurtis')

        
    


In [6]:

search.fit(X,y)

KNeighborsClassifier(algorithm='ball_tree', leaf_size=30, metric='braycurtis',
           metric_params=None, n_jobs=1, n_neighbors=10, p=2,
           weights='uniform')

In [7]:
valor = cross_val_score(search, X, y, scoring='roc_auc', cv=10)

valor.mean()


0.94207280200262833

In [8]:
print valor

[ 0.9591961   0.95774065  0.95889755  0.95439467  0.93127721  0.93333467
  0.92881766  0.93414814  0.93108693  0.93183444]


In [9]:
search2= KNeighborsClassifier(n_neighbors=1000, weights= 'distance', metric='braycurtis')
search2.fit(X,y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='braycurtis',
           metric_params=None, n_jobs=1, n_neighbors=1000, p=2,
           weights='distance')

In [10]:
valor2 = cross_val_score(search2, X, y, scoring='roc_auc', cv=10)

print valor2.mean(), valor

0.855627149352 [ 0.9672453   0.96627828  0.96761555  0.96345731  0.94566042  0.94751089
  0.9443788   0.94836762  0.94646967  0.94588001]


In [ ]:
search3= KNeighborsClassifier(n_neighbors=1000, weights= 'distance', metric='canberra')
search3.fit(X,y)


In [24]:

valor3 = cross_val_score(search3, X, y, scoring='roc_auc', cv=10)

print valor3.mean(), valor3

[ 0.9322123   0.93309812  0.93089563  0.92180642  0.88899595  0.90195462
  0.89488054  0.89699514  0.89299136  0.89859546]
